In [114]:
import pandas as pd
import numpy as np
import stat
import re
import warnings
warnings.filterwarnings("ignore")

In [117]:
cd 'cleaning'

/Users/ac/Documents/GitHub/txd-pipeline/data/classes/sp24/e29/check-in-1/cleaning


In [118]:
#analysis = pd.read_csv("FF_HEALTH_DC3_ANALYSIS.csv")
roster = pd.read_csv("FINAL_E29_ROSTER.CSV")
dictionary = pd.read_csv("E29_QUESTION_DICTIONARY.csv")
raw = pd.read_csv("E29_PRECLEAN_CHECKIN01_RAW_TEXT.csv")

full_cleaned_data = pd.read_csv("E29_CHECKIN01_FULL_CLEANED.csv")

In [119]:
full_cleaned_data

,Unnamed: 0,Email,TeamNumber,TeammateNumber,RecipientLastName,RecipientFirstName,RecipientEmail,32,33,33.4.TEXT,...,17.4,18.1,18.2,18.3,18.4,19.1,19.2,19.3,19.4,25
0,0,allanzhong@berkeley.edu,Team 1,1.0,Zhong,Allan,allanzhong@berkeley.edu,"Sometimes, it depends on the week and number o...",Virtual meetings,No Response,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,"No, I do not consent"
1,1,eclopez19@berkeley.edu,Team 1,2.0,Lopez,Edwin,eclopez19@berkeley.edu,"Rarely, we only meet to finalize the tasks",Other:,Currently through messages and facetime. Howev...,...,6.0,6.0,5.0,6.0,6.0,7.0,7.0,7.0,7.0,"Yes, I do consent"
2,2,jerrick@berkeley.edu,Team 1,3.0,Wang,Jerrick,jerrick@berkeley.edu,"Sometimes, it depends on the week and number o...","Asynchronous communication (i.e., texting, ema...",No Response,...,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,"Yes, I do consent"
3,3,simrankk@berkeley.edu,Team 1,4.0,Kaur Khahira,Simran,simrankk@berkeley.edu,"Rarely, we only meet to finalize the tasks",In-person meetings,No Response,...,5.0,7.0,5.0,7.0,7.0,5.0,4.0,5.0,5.0,"Yes, I do consent"
4,4,miroygarrett@berkeley.edu,Team 10,1.0,Garrett,Miro,miroygarrett@berkeley.edu,"Rarely, we only meet to finalize the tasks",In-person meetings,No Response,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,"Yes, I do consent"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,129,siddhant.vasudevan@berkeley.edu,Team 8,4.0,Vasudevan,Siddhant,siddhant.vasudevan@berkeley.edu,"Never, we divide the tasks asynchronously or i...","Asynchronous communication (i.e., texting, ema...",No Response,...,6.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,"Yes, I do consent"
130,130,allan.sram16@berkeley.edu,Team 9,1.0,Ramirez,Allan,allan.sram16@berkeley.edu,"Sometimes, it depends on the week and number o...","Asynchronous communication (i.e., texting, ema...",No Response,...,6.0,6.0,7.0,6.0,6.0,7.0,7.0,7.0,7.0,"Yes, I do consent"
131,131,arnavp45@berkeley.edu,Team 9,2.0,Patel,Arnav,arnavp45@berkeley.edu,"Sometimes, it depends on the week and number o...","Asynchronous communication (i.e., texting, ema...",No Response,...,5.0,7.0,7.0,4.0,6.0,7.0,7.0,7.0,7.0,"Yes, I do consent"
132,132,owenlachs@berkeley.edu,Team 9,3.0,Lachs,Owen,owenlachs@berkeley.edu,"Sometimes, it depends on the week and number o...",Virtual meetings,No Response,...,7.0,7.0,7.0,6.0,7.0,7.0,7.0,7.0,7.0,"Yes, I do consent"


In [120]:
quant = [str(x) for x in list(dictionary[dictionary["type"] == 'quantitative']["question_id"])]

numerical_data = full_cleaned_data[['TeamNumber', 'TeammateNumber','2.1',
 '2.2',
 '2.3',
 '4.1',
 '4.2',
 '4.3',
 '4.4',
 '5.1',
 '5.2',
 '5.3',
 '5.4',
 '16.1',
 '16.2',
 '16.3',
 '16.4',
 '17.1',
 '17.2',
 '17.3',
 '17.4',
 '18.1',
 '18.2',
 '18.3',
 '18.4',
 '19.1',
 '19.2',
 '19.3',
 '19.4']]
numerical_data = numerical_data.replace({"No Response" : np.nan})
numerical_data.head()

,TeamNumber,TeammateNumber,2.1,2.2,2.3,4.1,4.2,4.3,4.4,5.1,...,17.3,17.4,18.1,18.2,18.3,18.4,19.1,19.2,19.3,19.4
0,Team 1,1.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,25,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
1,Team 1,2.0,6.0,5.0,5.0,6.0,5.0,5.0,5.0,25,...,6.0,6.0,6.0,5.0,6.0,6.0,7.0,7.0,7.0,7.0
2,Team 1,3.0,5.0,6.0,5.0,5.0,5.0,6.0,6.0,28,...,5.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
3,Team 1,4.0,5.0,5.0,3.0,3.0,1.0,6.0,4.0,35,...,5.0,5.0,7.0,5.0,7.0,7.0,5.0,4.0,5.0,5.0
4,Team 10,1.0,5.0,5.0,6.0,6.0,7.0,5.0,7.0,25,...,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0


In [121]:
numerical_data["TeamNumber"] = [team.replace("Team", '') for team in numerical_data["TeamNumber"]]
numerical_data

,TeamNumber,TeammateNumber,2.1,2.2,2.3,4.1,4.2,4.3,4.4,5.1,...,17.3,17.4,18.1,18.2,18.3,18.4,19.1,19.2,19.3,19.4
0,1,1.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,25,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
1,1,2.0,6.0,5.0,5.0,6.0,5.0,5.0,5.0,25,...,6.0,6.0,6.0,5.0,6.0,6.0,7.0,7.0,7.0,7.0
2,1,3.0,5.0,6.0,5.0,5.0,5.0,6.0,6.0,28,...,5.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
3,1,4.0,5.0,5.0,3.0,3.0,1.0,6.0,4.0,35,...,5.0,5.0,7.0,5.0,7.0,7.0,5.0,4.0,5.0,5.0
4,10,1.0,5.0,5.0,6.0,6.0,7.0,5.0,7.0,25,...,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,8,4.0,6.0,6.0,7.0,6.0,6.0,6.0,7.0,25,...,6.0,6.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
130,9,1.0,6.0,4.0,5.0,6.0,6.0,6.0,6.0,30,...,6.0,6.0,6.0,7.0,6.0,6.0,7.0,7.0,7.0,7.0
131,9,2.0,6.0,5.0,6.0,5.0,6.0,6.0,5.0,30,...,6.0,5.0,7.0,7.0,4.0,6.0,7.0,7.0,7.0,7.0
132,9,3.0,6.0,5.0,6.0,6.0,6.0,6.0,6.0,26,...,6.0,7.0,7.0,7.0,6.0,7.0,7.0,7.0,7.0,7.0


In [122]:
numerical_data = numerical_data.astype(float)
numerical_data["Email"] = full_cleaned_data["Email"]

In [123]:
roster["TeamNumber"] = [team.replace("Team", '') for team in roster["TeamNumber"]]
full_cleaned_data["TeamNumber"] = [team.replace("Team", '') for team in full_cleaned_data["TeamNumber"]]


In [124]:
roster.head()

,Unnamed: 0,FullName,First name,Last Name,Email,TeamNumber,TeammateNumber,Teammate1,Teammate2,Teammate3,Teammate4
0,0,Allan Zhong,Allan,Zhong,allanzhong@berkeley.edu,1,1,Allan Zhong,Edwin Lopez,Jerrick Wang,Simran Kaur Khahira
1,1,Edwin Lopez,Edwin,Lopez,eclopez19@berkeley.edu,1,2,Allan Zhong,Edwin Lopez,Jerrick Wang,Simran Kaur Khahira
2,2,Jerrick Wang,Jerrick,Wang,jerrick@berkeley.edu,1,3,Allan Zhong,Edwin Lopez,Jerrick Wang,Simran Kaur Khahira
3,3,Simran Kaur Khahira,Simran,Kaur Khahira,simrankk@berkeley.edu,1,4,Allan Zhong,Edwin Lopez,Jerrick Wang,Simran Kaur Khahira
4,4,Miro Garrett,Miro,Garrett,miroygarrett@berkeley.edu,10,1,Miro Garrett,Taiki Yamamoto,Taylor Jazan,Toren Andersen


In [136]:
numerical_data

,TeamNumber,TeammateNumber,2.1,2.2,2.3,4.1,4.2,4.3,4.4,5.1,...,17.4,18.1,18.2,18.3,18.4,19.1,19.2,19.3,19.4,Email
0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,25.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,allanzhong@berkeley.edu
1,1.0,2.0,6.0,5.0,5.0,6.0,5.0,5.0,5.0,25.0,...,6.0,6.0,5.0,6.0,6.0,7.0,7.0,7.0,7.0,eclopez19@berkeley.edu
2,1.0,3.0,5.0,6.0,5.0,5.0,5.0,6.0,6.0,28.0,...,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,jerrick@berkeley.edu
3,1.0,4.0,5.0,5.0,3.0,3.0,1.0,6.0,4.0,35.0,...,5.0,7.0,5.0,7.0,7.0,5.0,4.0,5.0,5.0,simrankk@berkeley.edu
4,10.0,1.0,5.0,5.0,6.0,6.0,7.0,5.0,7.0,25.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,miroygarrett@berkeley.edu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,8.0,4.0,6.0,6.0,7.0,6.0,6.0,6.0,7.0,25.0,...,6.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,siddhant.vasudevan@berkeley.edu
130,9.0,1.0,6.0,4.0,5.0,6.0,6.0,6.0,6.0,30.0,...,6.0,6.0,7.0,6.0,6.0,7.0,7.0,7.0,7.0,allan.sram16@berkeley.edu
131,9.0,2.0,6.0,5.0,6.0,5.0,6.0,6.0,5.0,30.0,...,5.0,7.0,7.0,4.0,6.0,7.0,7.0,7.0,7.0,arnavp45@berkeley.edu
132,9.0,3.0,6.0,5.0,6.0,6.0,6.0,6.0,6.0,26.0,...,7.0,7.0,7.0,6.0,7.0,7.0,7.0,7.0,7.0,owenlachs@berkeley.edu


In [137]:
#Functions 

def calculate_team_averages(df):
    # Creates a DataFrame with the averages per team for each question
    return df.groupby("TeamNumber").agg(np.nanmean)[numerical_questions]

def get_team_data(team_number):
    # Returns new dataframe containing only TEAM_NUMBER's data
    return numerical_data[numerical_data['TeamNumber'] == float(team_number)]

def get_team_average(team_number, question):
    # Gets TEAM_NUMBER's average on the specified question
    temp_df = get_team_data(team_number)
    return np.nanmean(temp_df[question])

def get_team_std(team_number, teammate_number, question):
    # Gets TEAM_NUMBER's standard deviation on the specified question and should not include self
    temp_df = get_team_data(team_number)
    temp_df = temp_df[temp_df['TeammateNumber'] != teammate_number]
    
    return np.nanstd(temp_df[question])

def get_teammates_average(team_number, teammate_number, question):
    # Gets TEAM_NUMBER's average on the specified question
    # omits data from current student given by TEAMMATE_NUMBER 
    temp_df = get_team_data(team_number)
    
    temp_df = temp_df[temp_df['TeammateNumber'] != teammate_number]
    
    return np.nanmean(temp_df[question])


def get_student_answer(team_number, teammate_number, question):
    # Returns the current student's response for the specified question
    temp_df = get_team_data(team_number)
    
    temp_df = temp_df[temp_df['TeammateNumber'] == teammate_number]
    
    return float(temp_df[question])
    #return temp_df[question]

def get_class_average(question):
    # Returns the class average for QUESTION
    return np.mean(numerical_data[question])

def get_class_stdv(question):
    # Returns the class std dev for QUESTION
    return np.std(numerical_data[question])

## Part 1: Identification Information

In [138]:
#roster = roster.rename({"FullName": "Full Name"}, axis=1)

In [139]:
analysis_v2 = roster[["FullName", "Email", "TeamNumber", "TeammateNumber"]]

In [140]:
analysis_v2 = analysis_v2.sort_values(["TeamNumber", "TeammateNumber"], ascending=True).reset_index(drop=True)

In [141]:
analysis_v2

,FullName,Email,TeamNumber,TeammateNumber
0,Allan Zhong,allanzhong@berkeley.edu,1,1
1,Edwin Lopez,eclopez19@berkeley.edu,1,2
2,Jerrick Wang,jerrick@berkeley.edu,1,3
3,Simran Kaur Khahira,simrankk@berkeley.edu,1,4
4,Miro Garrett,miroygarrett@berkeley.edu,10,1
...,...,...,...,...
129,Siddhant Vasudevan,siddhant.vasudevan@berkeley.edu,8,4
130,Allan Ramirez,allan.sram16@berkeley.edu,9,1
131,Arnav Patel,arnavp45@berkeley.edu,9,2
132,Owen Lachs,owenlachs@berkeley.edu,9,3


## Part 2: Skills Assessment

In [142]:
...

Ellipsis

## Part 3: Survey Questionnairre

In [143]:
question_shorthand = {}
question_shorthand[2.1] = ['SharedGoalTeamAvg', 'SharedGoalClassAvg', 'SharedGoalClassStDev']
question_shorthand[2.2] = ['IndvGoalTeamAvg', 'IndvGoalClassAvg', 'IndvGoalClassStDev']
question_shorthand[2.3] = ['SupportTeamAvg', 'SupportClassAvg', 'SupportClassStDev']
question_shorthand[3.0] = ['Goal1', 'Goal2', 'Goal3', 'Goal4']
question_shorthand[4.1] = ['CommunicateTeamAvg', 'CommunicateClassAvg', 'CommunicateClassStDev']
question_shorthand[4.2] = ['WorkAllocTeamAvg', 'WorkAllocClassAvg', 'WorkAllocClassStDev']
question_shorthand[4.3] = ['RoleTeamAvg', 'RoleClassAvg', 'RoleClassStDev']
question_shorthand[4.4] = ['EnjoyTeamAvg', 'EnjoyClassAvg', 'EnjoyClassStDev']
question_shorthand[5.0] = ['MyPoints', 'TeammateAvgPoints', 'ExpectedPoints', 'Percent', 'AveragePointsAllocated', 'StDevPointsAllocated']
question_shorthand[5.1] = 'Work Contribution Teammate 1'
question_shorthand[5.2] = 'Work Contribution Teammate 2'
question_shorthand[5.3] = 'Work Contribution Teammate 3'
question_shorthand[5.4] = 'Work Contribution Teammate 4'
#question_shorthand[5.5] = 'Work Contribution Teammate 5'
#question_shorthand[5.6] = 'Work Contribution Teammate 6' #might need to increase this based on team size
question_shorthand[6.0] = ['DidWellMe','DidWell1','DidWell2','DidWell3','DidWell4']
question_shorthand[7] = ['DoDiffMe', 'DoDiff1', 'DoDiff2', 'DoDiff3', 'DoDiff4']
#question_shorthand[9.1] = 'ImproveAmt1'
#question_shorthand[9.2] = 'ImproveAmt2'
#question_shorthand[9.3] = 'ImproveAmt3'
#question_shorthand[10.1] = ["AskFeedbackClassAvg", "AskFeedbackClassStDev"]
#question_shorthand[10.2] = ["AskHelpClassAvg", "AskHelpClassStDev"]
#question_shorthand[10.3] = ["ValueContrClassAvg", "ValueContrClassStdev"]
#question_shorthand[10.4] = ["ReflectClassAvg", "ReflectClassStDev"]
#questions 11-13 are just imported as questions themselves as columns
question_shorthand[14] = ["ContributeMe", "Contribute1", "Contribute2", "Contribute3", "Contribute4"]
question_shorthand[15] = ['ImproveMe', 'Improve1', 'Improve2', 'Improve3', 'Improve4']
question_shorthand[16] = ['InitiativeMe', 'InitiativeTeammateAvg', 'InitiativeClassAvg', 'InitiativeClassStDev']
question_shorthand[17] = ['CommunicateMe', 'CommunicateTeammateAvg', 'Communicate2ClassAvg', 'Communicate2ClassStDev']
question_shorthand[18] = ['ExpertiseMe', 'ExpertiseTeammateAvg', 'ExpertiseClassAvg', 'ExpertiseClassStDev']
question_shorthand[19] = ['RespectMe', 'RespectTeammateAvg', 'RespectClassAvg', 'RespectClassStDev']
#question_shorthand[38.1] = 'ImproveAmt1'
#question_shorthand[38.2] = 'ImproveAmt2'
#question_shorthand[38.3] = 'ImproveAmt3'

### 2.1

In [144]:
#Question 2.1

question_2_1 = dictionary[dictionary["question_id"] == 2.1].reset_index()["question"][0]
analysis_v2[question_2_1] = numerical_data["2.1"]

In [145]:
#Question 2.1 team average
question_2_1_0 = question_shorthand[2.1][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "2.1"), 1))

analysis_v2[question_2_1_0] = lst_responses

In [146]:
analysis_v2

,FullName,Email,TeamNumber,TeammateNumber,Our team is clear about the shared goals for our work together,SharedGoalTeamAvg
0,Allan Zhong,allanzhong@berkeley.edu,1,1,6.0,5.5
1,Edwin Lopez,eclopez19@berkeley.edu,1,2,6.0,5.5
2,Jerrick Wang,jerrick@berkeley.edu,1,3,5.0,5.5
3,Simran Kaur Khahira,simrankk@berkeley.edu,1,4,5.0,5.5
4,Miro Garrett,miroygarrett@berkeley.edu,10,1,5.0,6.0
...,...,...,...,...,...,...
129,Siddhant Vasudevan,siddhant.vasudevan@berkeley.edu,8,4,6.0,6.2
130,Allan Ramirez,allan.sram16@berkeley.edu,9,1,6.0,6.0
131,Arnav Patel,arnavp45@berkeley.edu,9,2,6.0,6.0
132,Owen Lachs,owenlachs@berkeley.edu,9,3,6.0,6.0


In [147]:
#Question 2.1 class avg
question_2_1_1 = question_shorthand[2.1][1]

analysis_v2[question_2_1_1] = round(get_class_average("2.1"), 1)

In [148]:


#Question 2.1 class std
question_2_1_2 = question_shorthand[2.1][2]

analysis_v2[question_2_1_2] = round(get_class_stdv("2.1"), 1)

In [149]:
analysis_v2

,FullName,Email,TeamNumber,TeammateNumber,Our team is clear about the shared goals for our work together,SharedGoalTeamAvg,SharedGoalClassAvg,SharedGoalClassStDev
0,Allan Zhong,allanzhong@berkeley.edu,1,1,6.0,5.5,6.0,0.9
1,Edwin Lopez,eclopez19@berkeley.edu,1,2,6.0,5.5,6.0,0.9
2,Jerrick Wang,jerrick@berkeley.edu,1,3,5.0,5.5,6.0,0.9
3,Simran Kaur Khahira,simrankk@berkeley.edu,1,4,5.0,5.5,6.0,0.9
4,Miro Garrett,miroygarrett@berkeley.edu,10,1,5.0,6.0,6.0,0.9
...,...,...,...,...,...,...,...,...
129,Siddhant Vasudevan,siddhant.vasudevan@berkeley.edu,8,4,6.0,6.2,6.0,0.9
130,Allan Ramirez,allan.sram16@berkeley.edu,9,1,6.0,6.0,6.0,0.9
131,Arnav Patel,arnavp45@berkeley.edu,9,2,6.0,6.0,6.0,0.9
132,Owen Lachs,owenlachs@berkeley.edu,9,3,6.0,6.0,6.0,0.9


### 2.2

In [150]:
#Question 2.2

question_2_2 = dictionary[dictionary["question_id"] == 2.2].reset_index()["question"][0]
analysis_v2[question_2_2] = numerical_data["2.2"]

In [151]:
#Question 2.2 team average
question_2_2_0 = question_shorthand[2.2][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "2.2"), 1))

analysis_v2[question_2_2_0] = lst_responses

In [152]:
#Question 2.2 class avg
question_2_2_1 = question_shorthand[2.2][1]

analysis_v2[question_2_2_1] = round(get_class_average("2.2"), 1)

In [153]:
#Question 2.2 class std
question_2_2_2 = question_shorthand[2.2][2]

analysis_v2[question_2_2_2] = round(get_class_stdv("2.2"), 1)

In [154]:
analysis_v2

,FullName,Email,TeamNumber,TeammateNumber,Our team is clear about the shared goals for our work together,SharedGoalTeamAvg,SharedGoalClassAvg,SharedGoalClassStDev,We each know about one another's individual goals for our work together,IndvGoalTeamAvg,IndvGoalClassAvg,IndvGoalClassStDev
0,Allan Zhong,allanzhong@berkeley.edu,1,1,6.0,5.5,6.0,0.9,6.0,5.5,5.8,1.1
1,Edwin Lopez,eclopez19@berkeley.edu,1,2,6.0,5.5,6.0,0.9,5.0,5.5,5.8,1.1
2,Jerrick Wang,jerrick@berkeley.edu,1,3,5.0,5.5,6.0,0.9,6.0,5.5,5.8,1.1
3,Simran Kaur Khahira,simrankk@berkeley.edu,1,4,5.0,5.5,6.0,0.9,5.0,5.5,5.8,1.1
4,Miro Garrett,miroygarrett@berkeley.edu,10,1,5.0,6.0,6.0,0.9,5.0,6.0,5.8,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...
129,Siddhant Vasudevan,siddhant.vasudevan@berkeley.edu,8,4,6.0,6.2,6.0,0.9,6.0,6.2,5.8,1.1
130,Allan Ramirez,allan.sram16@berkeley.edu,9,1,6.0,6.0,6.0,0.9,4.0,4.8,5.8,1.1
131,Arnav Patel,arnavp45@berkeley.edu,9,2,6.0,6.0,6.0,0.9,5.0,4.8,5.8,1.1
132,Owen Lachs,owenlachs@berkeley.edu,9,3,6.0,6.0,6.0,0.9,5.0,4.8,5.8,1.1


### 2.3

In [155]:
#Question 2.3

question_2_3 = dictionary[dictionary["question_id"] == 2.3].reset_index()["question"][0]
analysis_v2[question_2_3] = numerical_data["2.3"]

In [156]:
#Question 2.3 team average
question_2_3_0 = question_shorthand[2.3][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "2.3"), 1))

analysis_v2[question_2_3_0] = lst_responses

In [157]:
#Question 2.3 class avg
question_2_3_1 = question_shorthand[2.3][1]

analysis_v2[question_2_3_1] = round(get_class_average("2.3"), 1)

In [158]:
#Question 2.3 class std
question_2_3_2 = question_shorthand[2.3][2]

analysis_v2[question_2_3_2] = round(get_class_stdv("2.3"), 1)

### 3

In [159]:
question_3 = dictionary[dictionary["question_id"] == 3.0].reset_index()["question"][0]

In [160]:
analysis_v2[question_3] = full_cleaned_data["3"]

In [161]:
#goal 1
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 1].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_1 = question_shorthand[3.0][0]
analysis_v2[question_3_1] = lst_responses

In [162]:
#goal 2
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 2].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_2 = question_shorthand[3.0][1]
analysis_v2[question_3_2] = lst_responses

In [163]:
#goal 3
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 3].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_3 = question_shorthand[3.0][2]
analysis_v2[question_3_3] = lst_responses

In [164]:
#goal 4
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 4].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_4 = question_shorthand[3.0][3]
analysis_v2[question_3_4] = lst_responses

### 4.1

In [165]:
#Question 4.1

question_4_1 = dictionary[dictionary["question_id"] == 4.1].reset_index()["question"][0]
analysis_v2[question_4_1] = numerical_data["4.1"]

In [166]:
#analysis_v2

In [167]:
#Question 4.1 team average
question_4_1_0 = question_shorthand[4.1][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.1"), 1))

analysis_v2[question_4_1_0] = lst_responses

In [168]:
#analysis_v2

In [169]:
#Question 4.1 class avg
question_4_1_1 = question_shorthand[4.1][1]

analysis_v2[question_4_1_1] = round(get_class_average("4.1"), 1)

In [170]:
#Question 4.1 class std
question_4_1_2 = question_shorthand[4.1][2]

analysis_v2[question_4_1_2] = round(get_class_stdv("4.1"), 1)

### 4.2

In [171]:
#Question 4.2

question_4_2 = dictionary[dictionary["question_id"] == 4.2].reset_index()["question"][0]
analysis_v2[question_4_2] = numerical_data["4.2"]

In [172]:
#Question 4.2 team average
question_4_2_0 = question_shorthand[4.2][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.2"), 1))

analysis_v2[question_4_2_0] = lst_responses

In [173]:
#Question 4.2 class avg
question_4_2_1 = question_shorthand[4.2][1]

analysis_v2[question_4_2_1] = round(get_class_average("4.2"), 1)

In [174]:
#Question 4.2 class std
question_4_2_2 = question_shorthand[4.2][2]

analysis_v2[question_4_2_2] = round(get_class_stdv("4.2"), 1)

### 4.3

In [175]:
#Question 4.3

question_4_3 = dictionary[dictionary["question_id"] == 4.3].reset_index()["question"][0]
analysis_v2[question_4_3] = numerical_data["4.3"]

In [176]:
#Question 4.1 team average
question_4_3_0 = question_shorthand[4.3][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.3"), 1))

analysis_v2[question_4_3_0] = lst_responses

In [177]:
#Question 4.3 class avg
question_4_3_1 = question_shorthand[4.3][1]

analysis_v2[question_4_3_1] = round(get_class_average("4.3"), 1)

In [178]:
#Question 4.3 class std
question_4_3_2 = question_shorthand[4.3][2]

analysis_v2[question_4_3_2] = round(get_class_stdv("4.3"), 1)

### 4.4

In [179]:
#Question 4.4

question_4_4 = dictionary[dictionary["question_id"] == 4.4].reset_index()["question"][0]
analysis_v2[question_4_4] = numerical_data["4.4"]

In [180]:
#Question 4.4 team average
question_4_4_0 = question_shorthand[4.4][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.4"), 1))

analysis_v2[question_4_4_0] = lst_responses

In [181]:
#Question 4.4 class avg
question_4_4_1 = question_shorthand[4.4][1]

analysis_v2[question_4_4_1] = round(get_class_average("4.4"), 1)

In [182]:
#Question 4.4 class std
question_4_4_2 = question_shorthand[4.4][2]

analysis_v2[question_4_4_2] = round(get_class_stdv("4.4"), 1)

### 5

In [183]:
#Question 5.1-5.6. will need to change this according to how many people are on a team
question_5_1 = dictionary[dictionary["question_id"] == 5.1].reset_index()["question"][0]
question_5_2 = dictionary[dictionary["question_id"] == 5.2].reset_index()["question"][0]
question_5_3 = dictionary[dictionary["question_id"] == 5.3].reset_index()["question"][0]
question_5_4 = dictionary[dictionary["question_id"] == 5.4].reset_index()["question"][0]
#question_5_5 = dictionary[dictionary["question_id"] == 5.5].reset_index()["question"][0]
#question_5_6 = dictionary[dictionary["question_id"] == 5.6].reset_index()["question"][0]

analysis_v2[question_5_1] = full_cleaned_data[["5.1"]]
analysis_v2[question_5_2] = full_cleaned_data[["5.2"]]
analysis_v2[question_5_3] = full_cleaned_data[["5.3"]]
analysis_v2[question_5_4] = full_cleaned_data[["5.4"]]
#analysis_v2[question_5_5] = full_cleaned_data[["5.5"]]
#analysis_v2[question_5_6] = full_cleaned_data[["5.6"]]

In [184]:
#Calculations

#MyPoints
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    student_df = team_df[team_df["TeammateNumber"] == student_teammate_number].reset_index()
    question_needed = '5.' + str(int(student_teammate_number))
    #print(student_df)
    lst_responses.append(student_df[question_needed].reset_index()[question_needed][0])
    
question_5_a = question_shorthand[5][0]
analysis_v2[question_5_a] = lst_responses


In [185]:
#TeammateAvgPoints
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    question_needed = '5.' + str(int(student_teammate_number))
    lst_responses.append(round(get_teammates_average(student_team_number, student_teammate_number, question_needed), 1))

question_5_b = question_shorthand[5][1]
analysis_v2[question_5_b] = lst_responses



In [186]:
#ExpectedPoints
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    lst_responses.append(round(100/team_size, 1))

question_5_c = question_shorthand[5][2]
analysis_v2[question_5_c] = lst_responses

In [187]:
#Percent
question_5_d = question_shorthand[5][3]
analysis_v2[question_5_d] = round((analysis_v2[question_5_b] / analysis_v2[question_5_c]) * 100, 1)

In [188]:
#AveragePointsAllocated
lst_responses = []
for student_email in analysis_v2["Email"]:
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    
    temp_q5_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number][["5.1", "5.2", "5.3", "5.4"]]
    temp_q5_df = temp_q5_df.replace({"No Response" : np.nan}).astype(float)
    sum_points = sum(temp_q5_df.reset_index().drop("index", axis=1).iloc[int(student_teammate_number) - 1])
    lst_responses.append(round(sum_points/team_size, 1))

question_5_e = question_shorthand[5][4]
analysis_v2[question_5_e] = lst_responses

In [189]:
#StDevPointsAllocated
lst_responses = []
for student_email in analysis_v2["Email"]:
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    
    temp_q5_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number][["5.1", "5.2", "5.3", "5.4"]]
    temp_q5_df = temp_q5_df.replace({"No Response" : np.nan}).astype(float)
    arr = temp_q5_df.reset_index().drop("index", axis=1).dropna().to_numpy().flatten()
    std = np.std(arr[arr != 0])
    lst_responses.append(round(std, 1))

question_5_f = question_shorthand[5][5]
analysis_v2[question_5_f] = lst_responses

In [190]:
analysis_v2

,FullName,Email,TeamNumber,TeammateNumber,Our team is clear about the shared goals for our work together,SharedGoalTeamAvg,SharedGoalClassAvg,SharedGoalClassStDev,We each know about one another's individual goals for our work together,IndvGoalTeamAvg,...,Please allocate 100 points across yourself and your teammates for your contributions to the work of the team. For teammate 1,Please allocate 100 points across yourself and your teammates for your contributions to the work of the team. For teammate 2,Please allocate 100 points across yourself and your teammates for your contributions to the work of the team. For teammate 3,Please allocate 100 points across yourself and your teammates for your contributions to the work of the team. For teammate 4,MyPoints,TeammateAvgPoints,ExpectedPoints,Percent,AveragePointsAllocated,StDevPointsAllocated
0,Allan Zhong,allanzhong@berkeley.edu,1,1,6.0,5.5,6.0,0.9,6.0,5.5,...,25,25,25,25,25,29.3,25.0,117.2,25.0,4.8
1,Edwin Lopez,eclopez19@berkeley.edu,1,2,6.0,5.5,6.0,0.9,5.0,5.5,...,25,15,30,30,15,20.7,25.0,82.8,25.0,4.8
2,Jerrick Wang,jerrick@berkeley.edu,1,3,5.0,5.5,6.0,0.9,6.0,5.5,...,28,22,25,25,25,26.7,25.0,106.8,25.0,4.8
3,Simran Kaur Khahira,simrankk@berkeley.edu,1,4,5.0,5.5,6.0,0.9,5.0,5.5,...,35,15,25,25,25,26.7,25.0,106.8,25.0,4.8
4,Miro Garrett,miroygarrett@berkeley.edu,10,1,5.0,6.0,6.0,0.9,5.0,6.0,...,25,25,25,25,25,23.0,25.0,92.0,25.0,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,Siddhant Vasudevan,siddhant.vasudevan@berkeley.edu,8,4,6.0,6.2,6.0,0.9,6.0,6.2,...,25,25,25,25,25,25.3,25.0,101.2,25.0,0.9
130,Allan Ramirez,allan.sram16@berkeley.edu,9,1,6.0,6.0,6.0,0.9,4.0,4.8,...,30,30,10,30,30,27.0,25.0,108.0,25.0,5.3
131,Arnav Patel,arnavp45@berkeley.edu,9,2,6.0,6.0,6.0,0.9,5.0,4.8,...,30,30,15,25,30,27.0,25.0,108.0,25.0,5.3
132,Owen Lachs,owenlachs@berkeley.edu,9,3,6.0,6.0,6.0,0.9,5.0,4.8,...,26,26,23,25,23,16.7,25.0,66.8,25.0,5.3


### 6

In [191]:
#Didwellme 

lst_responses = []
for student_email in analysis_v2["Email"]:
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    temp_q6_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    response = temp_q6_df[temp_q6_df["TeammateNumber"] == student_teammate_number].reset_index()["6"][0]
    lst_responses.append(response)

question_6_a = question_shorthand[6][0]
analysis_v2[question_6_a] = lst_responses

In [192]:
#DidWell1 thru DidWell6

subset_df = analysis_v2[["TeamNumber", "TeammateNumber", "DidWellMe"]]
#subset_df[subset_df["TeammateNumber"] == 1]
#subset_df

lst_responses_1 = []
lst_responses_2 = []
lst_responses_3 = []
lst_responses_4 = []
#lst_responses_5 = []
#lst_responses_6 = []

for team in subset_df["TeamNumber"]:#.unique():
    team_series = subset_df[subset_df["TeamNumber"] == team].reset_index()["DidWellMe"]
    if len(team_series) == 2:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(np.nan)
        lst_responses_4.append(np.nan)
        #lst_responses_5.append(np.nan)
        #lst_responses_6.append(np.nan)
    if len(team_series) == 3:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(np.nan)
        #lst_responses_5.append(np.nan)
        #lst_responses_6.append(np.nan)
    if len(team_series) == 4:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        #lst_responses_5.append(np.nan)
        #lst_responses_6.append(np.nan)
    if len(team_series) == 5:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        #lst_responses_5.append(team_series[4])
        #lst_responses_6.append(np.nan)
    if len(team_series) == 6:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
       # lst_responses_5.append(team_series[4])
       # lst_responses_6.append(team_series[5])

question_6_b = question_shorthand[6][1]
question_6_c = question_shorthand[6][2]
question_6_d = question_shorthand[6][3]
question_6_e = question_shorthand[6][4]
#question_6_f = question_shorthand[6][5]
analysis_v2[question_6_b] = lst_responses_1
analysis_v2[question_6_c] = lst_responses_2
analysis_v2[question_6_d] = lst_responses_3
analysis_v2[question_6_e] = lst_responses_4
#analysis_v2[question_6_f] = lst_responses_5
#analysis_v2["DidWell6"] = lst_responses_6

### 7

In [193]:
#DoDiffMe
lst_responses = []
for student_email in analysis_v2["Email"]:
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    temp_q7_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    #temp_q7_df = temp_q7_df.replace({"No Response" : np.nan}).astype(float)
    response = temp_q7_df[temp_q7_df["TeammateNumber"] == student_teammate_number].reset_index()["7"][0]
    lst_responses.append(response)

question_7_a = question_shorthand[7][0]
analysis_v2[question_7_a] = lst_responses

In [194]:
#DoDiff1 thru DoDiff6

subset_df = analysis_v2[["TeamNumber", "TeammateNumber", "DoDiffMe"]]
#subset_df[subset_df["TeammateNumber"] == 1]
#subset_df

lst_responses_1 = []
lst_responses_2 = []
lst_responses_3 = []
lst_responses_4 = []
#lst_responses_5 = []
#lst_responses_6 = []

for team in subset_df["TeamNumber"]:#.unique():
    team_series = subset_df[subset_df["TeamNumber"] == team].reset_index()["DoDiffMe"]
    if len(team_series) == 2:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(np.nan)
        lst_responses_4.append(np.nan)
        #lst_responses_5.append(np.nan)
        #lst_responses_6.append(np.nan)
    if len(team_series) == 3:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(np.nan)
        #lst_responses_5.append(np.nan)
        #lst_responses_6.append(np.nan)
    if len(team_series) == 4:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        #lst_responses_5.append(np.nan)
        #lst_responses_6.append(np.nan)
    if len(team_series) == 5:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        #lst_responses_5.append(team_series[4])
        #lst_responses_6.append(np.nan)
    if len(team_series) == 6:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
       # lst_responses_5.append(team_series[4])
       # lst_responses_6.append(team_series[5])
        
question_7_b = question_shorthand[7][1]
question_7_c = question_shorthand[7][2]
question_7_d = question_shorthand[7][3]
question_7_e = question_shorthand[7][4]
#question_7_f = question_shorthand[7][5]
analysis_v2[question_7_b] = lst_responses_1
analysis_v2[question_7_c] = lst_responses_2
analysis_v2[question_7_d] = lst_responses_3
analysis_v2[question_7_e] = lst_responses_4
#analysis_v2[question_7_f] = lst_responses_5
#analysis_v2["DoDiff6"] = lst_responses_6

### 9

In [195]:
#analysis_v2[question_shorthand[9.1]] = full_cleaned_data["9.1"]
#analysis_v2[question_shorthand[9.2]] = full_cleaned_data["9.2"]
#analysis_v2[question_shorthand[9.3]] = full_cleaned_data["9.3"]

### 10 (not used here but can use functions below)

In [196]:
#analysis_v2[question_shorthand[10.1][0]] = get_class_average("10.1")
#analysis_v2[question_shorthand[10.1][1]] = get_class_stdv("10.1")

#analysis_v2[question_shorthand[10.2][0]] = get_class_average("10.2")
#analysis_v2[question_shorthand[10.2][1]] = get_class_stdv("10.2")

#analysis_v2[question_shorthand[10.3][0]] = get_class_average("10.3")
#analysis_v2[question_shorthand[10.3][1]] = get_class_stdv("10.3")


### 11

In [197]:
#question_11 = dictionary[dictionary["question_id"] == 11].reset_index()["question"][0]
#analysis_v2[question_11] = full_cleaned_data["11"]

### 12

In [198]:
#question_12 = dictionary[dictionary["question_id"] == 12].reset_index()["question"][0]
#analysis_v2[question_12] = full_cleaned_data["12"]

### 14

In [199]:
#14.1 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '14.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_14_me = question_shorthand[14][0]
analysis_v2[question_14_me] = lst_responses

In [200]:
#14

question_14_1 = question_shorthand[14][1]
question_14_2 = question_shorthand[14][2]
question_14_3 = question_shorthand[14][3]
question_14_4 = question_shorthand[14][4]
#question_14_5 = question_shorthand[14][5]
#question_14_6 = question_shorthand[14][6]

feedback_1 = []
feedback_2 = []
feedback_3 = []
feedback_4 = []
#feedback_5 = [] 
#feedback_6 = []#need to adjust to max team size

#possibilities = ["14.1", "14.2", "14.3", "14.4", "14.5", "14.6"]
for i in range(len(full_cleaned_data)):
    my_teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    my_team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    my_team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == my_team_number]
    list_feedback_for_me = list(my_team_df["14." + str(my_teammate_number)])
    num_teammates = len(list_feedback_for_me)
    extra_needed = 6 - num_teammates
    placeholders = [''] * extra_needed
    all_feedback = list_feedback_for_me + placeholders

    feedback_1.append(all_feedback[0])
    feedback_2.append(all_feedback[1])
    feedback_3.append(all_feedback[2])
    feedback_4.append(all_feedback[3])
    #feedback_5.append(all_feedback[4])
    #feedback_6.append(all_feedback[5])
    
analysis_v2[question_14_1] = feedback_1
analysis_v2[question_14_2] = feedback_2
analysis_v2[question_14_3] = feedback_3
analysis_v2[question_14_4] = feedback_4
#analysis_v2[question_14_5] = feedback_5
#analysis_v2[question_14_6] = feedback_6

### 15

In [201]:
#15.1 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '15.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_15_me = question_shorthand[15][0]
analysis_v2[question_15_me] = lst_responses

In [202]:
#15

question_15_1 = question_shorthand[15][1]
question_15_2 = question_shorthand[15][2]
question_15_3 = question_shorthand[15][3]
question_15_4 = question_shorthand[15][4]
#question_15_5 = question_shorthand[15][5]
#question_15_6 = question_shorthand[15][6]

feedback_1 = []
feedback_2 = []
feedback_3 = []
feedback_4 = []
#feedback_5 = [] 
#feedback_6 = []#need to adjust to max team size

#possibilities = ["14.1", "14.2", "14.3", "14.4", "14.5", "14.6"]
for i in range(len(full_cleaned_data)):
    my_teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    my_team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    my_team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == my_team_number]
    list_feedback_for_me = list(my_team_df["15." + str(my_teammate_number)])
    num_teammates = len(list_feedback_for_me)
    extra_needed = 6 - num_teammates
    placeholders = [''] * extra_needed
    all_feedback = list_feedback_for_me + placeholders

    feedback_1.append(all_feedback[0])
    feedback_2.append(all_feedback[1])
    feedback_3.append(all_feedback[2])
    feedback_4.append(all_feedback[3])
    #feedback_5.append(all_feedback[4])
    #feedback_6.append(all_feedback[5])
    
analysis_v2[question_15_1] = feedback_1
analysis_v2[question_15_2] = feedback_2
analysis_v2[question_15_3] = feedback_3
analysis_v2[question_15_4] = feedback_4
#analysis_v2[question_15_5] = feedback_5
#analysis_v2[question_15_6] = feedback_6

### 16

In [203]:
#16 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '16.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_16_me = question_shorthand[16][0]
analysis_v2[question_16_me] = lst_responses

In [204]:
#Question 16 team average
question_16_tm_avg = question_shorthand[16][1]
lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    question_needed = '16.' + str(teammate_number)
    lst_responses.append(round(get_teammates_average(team_number, teammate_number, question_needed), 1))

analysis_v2[question_16_tm_avg] = lst_responses

In [205]:
#Question 16 class avg
question_16_class_avg = question_shorthand[16][2]
class_avg = np.nanmean(full_cleaned_data[["16.1", "16.2", "16.3", "16.4"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten())
analysis_v2[question_16_class_avg] = round(class_avg, 1)

In [206]:
#Question 16 class std
question_16_class_std = question_shorthand[16][3]
class_std_arr = full_cleaned_data[["16.1", "16.2", "16.3", "16.4"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten()
class_std_arr_no_na = class_std_arr[~np.isnan(class_std_arr)]
analysis_v2[question_16_class_std] = round(np.std(class_std_arr_no_na), 1)

### 17

In [207]:
#17 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '17.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_17_me = question_shorthand[17][0]
analysis_v2[question_17_me] = lst_responses

In [208]:
#Question 17 team average
question_17_tm_avg = question_shorthand[17][1]
lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    question_needed = '17.' + str(teammate_number)
    lst_responses.append(round(get_teammates_average(team_number, teammate_number, question_needed), 1))

analysis_v2[question_17_tm_avg] = lst_responses

In [209]:
#analysis_v2

In [210]:
#Question 17 class avg
question_17_class_avg = question_shorthand[17][2]
class_avg = np.nanmean(full_cleaned_data[["17.1", "17.2", "17.3", "17.4"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten())
analysis_v2[question_17_class_avg] = round(class_avg, 1)

In [211]:
#Question 17 class std
question_17_class_std = question_shorthand[17][3]
class_std_arr = full_cleaned_data[["17.1", "17.2", "17.3", "17.4"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten()
class_std_arr_no_na = class_std_arr[~np.isnan(class_std_arr)]
analysis_v2[question_17_class_std] = round(np.std(class_std_arr_no_na), 1)

### 18

In [212]:
#18 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '18.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_18_me = question_shorthand[18][0]
analysis_v2[question_18_me] = lst_responses

In [213]:
#Question 18 team average
question_18_tm_avg = question_shorthand[18][1]
lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    question_needed = '18.' + str(teammate_number)
    lst_responses.append(round(get_teammates_average(team_number, teammate_number, question_needed), 1))

analysis_v2[question_18_tm_avg] = lst_responses

In [214]:
#Question 18 class avg
question_18_class_avg = question_shorthand[18][2]
class_avg = np.nanmean(full_cleaned_data[["18.1", "18.2", "18.3", "18.4"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten())
analysis_v2[question_18_class_avg] = round(class_avg, 1)

In [215]:
#Question 18 class std
question_18_class_std = question_shorthand[18][3]
class_std_arr = full_cleaned_data[["18.1", "18.2", "18.3", "18.4"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten()
class_std_arr_no_na = class_std_arr[~np.isnan(class_std_arr)]
analysis_v2[question_18_class_std] = round(np.std(class_std_arr_no_na), 1)

### 19

In [216]:
#19 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '19.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_19_me = question_shorthand[19][0]
analysis_v2[question_19_me] = lst_responses

In [217]:
#Question 19 team average
question_19_tm_avg = question_shorthand[19][1]
lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    question_needed = '19.' + str(teammate_number)
    lst_responses.append(round(get_teammates_average(team_number, teammate_number, question_needed), 1))

analysis_v2[question_19_tm_avg] = lst_responses

In [218]:
#Question 19 class avg
question_19_class_avg = question_shorthand[19][2]
class_avg = np.nanmean(full_cleaned_data[["19.1", "19.2", "19.3", "19.4"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten())
analysis_v2[question_19_class_avg] = round(class_avg, 1)

In [219]:
#Question 19 class std
question_19_class_std = question_shorthand[19][3]
class_std_arr = full_cleaned_data[["19.1", "19.2", "19.3", "19.4"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten()
class_std_arr_no_na = class_std_arr[~np.isnan(class_std_arr)]
analysis_v2[question_19_class_std] = round(np.std(class_std_arr_no_na), 1)

## Add extra questions

In [220]:
analysis_v2["Q20_1"] = full_cleaned_data['20.1']
analysis_v2["Q20_2"] = full_cleaned_data['20.2']
analysis_v2["Q20_3"] = full_cleaned_data['20.3']
analysis_v2["Q20_4"] = full_cleaned_data['20.4']
analysis_v2["Q20_5"] = full_cleaned_data['20.5']
analysis_v2["Q20_6"] = full_cleaned_data['20.6']
analysis_v2["Q20_7"] = full_cleaned_data['20.7']
analysis_v2["Q20_8"] = full_cleaned_data['20.8']
analysis_v2["Q20_9"] = full_cleaned_data['20.9']
analysis_v2["Q20_10"] = full_cleaned_data['20.10']
analysis_v2["Q20_11"] = full_cleaned_data['20.11']
analysis_v2["Q20_12"] = full_cleaned_data['20.12']
analysis_v2["Q20_13"] = full_cleaned_data['20.13']
analysis_v2["Q20_14"] = full_cleaned_data['20.14']
analysis_v2["Q20_15"] = full_cleaned_data['20.15']
analysis_v2["Q20_16"] = full_cleaned_data['20.16']
analysis_v2["Q20_17"] = full_cleaned_data['20.17']
analysis_v2["Q20_18"] = full_cleaned_data['20.18']
analysis_v2["Q20_19"] = full_cleaned_data['20.19']
analysis_v2["Q20_20"] = full_cleaned_data['20.20']
analysis_v2["Q21_1"] = full_cleaned_data['21.1']
analysis_v2["Q21_2"] = full_cleaned_data['21.2']
analysis_v2["Q21_3"] = full_cleaned_data['21.3']

analysis_v2["Q33"] = full_cleaned_data['33']
analysis_v2["Q33_TEXT"] = full_cleaned_data['33.4.TEXT']
analysis_v2["Q25"] = full_cleaned_data['25']

In [221]:
analysis_v2

,FullName,Email,TeamNumber,TeammateNumber,Our team is clear about the shared goals for our work together,SharedGoalTeamAvg,SharedGoalClassAvg,SharedGoalClassStDev,We each know about one another's individual goals for our work together,IndvGoalTeamAvg,...,Q20_17,Q20_18,Q20_19,Q20_20,Q21_1,Q21_2,Q21_3,Q33,Q33_TEXT,Q25
0,Allan Zhong,allanzhong@berkeley.edu,1,1,6.0,5.5,6.0,0.9,6.0,5.5,...,Always,Always,Always,Always,I encourage others on the team to contribute,"I help my team adhere to schedules, e.g., enga...",I help my teammates recognize the value of the...,Virtual meetings,No Response,"No, I do not consent"
1,Edwin Lopez,eclopez19@berkeley.edu,1,2,6.0,5.5,6.0,0.9,5.0,5.5,...,Always,Most of the time,About half the time,Always,I do a fair share of the work,I communicate information and feelings clearly,I facilitate exchange of ideas among my teammates,Other:,Currently through messages and facetime. Howev...,"Yes, I do consent"
2,Jerrick Wang,jerrick@berkeley.edu,1,3,5.0,5.5,6.0,0.9,6.0,5.5,...,Most of the time,About half the time,About half the time,Most of the time,I communicate information and feelings clearly,"I help my team adhere to schedules, e.g., enga...",I help my teammates recognize the value of the...,"Asynchronous communication (i.e., texting, ema...",No Response,"Yes, I do consent"
3,Simran Kaur Khahira,simrankk@berkeley.edu,1,4,5.0,5.5,6.0,0.9,5.0,5.5,...,Most of the time,Always,About half the time,Most of the time,I teach my teammates how to do elements of the...,I encourage my team to set aside time for feed...,I help my teammates recognize the value of the...,In-person meetings,No Response,"Yes, I do consent"
4,Miro Garrett,miroygarrett@berkeley.edu,10,1,5.0,6.0,6.0,0.9,5.0,6.0,...,Always,Most of the time,Most of the time,Most of the time,I ask my teammates for help when I need it,I encourage my team to set high goals and sust...,I help my teammates recognize the value of the...,In-person meetings,No Response,"Yes, I do consent"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,Siddhant Vasudevan,siddhant.vasudevan@berkeley.edu,8,4,6.0,6.2,6.0,0.9,6.0,6.2,...,Always,About half the time,Sometimes,Always,I ask my teammates for feedback on how I can i...,"I help my team adhere to schedules, e.g., enga...",I encourage my team to set aside time for feed...,"Asynchronous communication (i.e., texting, ema...",No Response,"Yes, I do consent"
130,Allan Ramirez,allan.sram16@berkeley.edu,9,1,6.0,6.0,6.0,0.9,4.0,4.8,...,Always,Most of the time,Most of the time,Most of the time,"I prepare for, arrive on time for and fully pa...",I pay attention to whether our team as a whole...,I encourage my team to set high goals and sust...,"Asynchronous communication (i.e., texting, ema...",No Response,"Yes, I do consent"
131,Arnav Patel,arnavp45@berkeley.edu,9,2,6.0,6.0,6.0,0.9,5.0,4.8,...,Most of the time,Most of the time,"Most of the time,Sometimes",Always,I teach my teammates how to do elements of the...,I ask my teammates for feedback on how I can i...,I encourage my team to set aside time for feed...,"Asynchronous communication (i.e., texting, ema...",No Response,"Yes, I do consent"
132,Owen Lachs,owenlachs@berkeley.edu,9,3,6.0,6.0,6.0,0.9,5.0,4.8,...,Most of the time,Always,Always,Always,I keep my commitments,I respond promptly to communication from my te...,I encourage my team to set aside time for feed...,Virtual meetings,No Response,"Yes, I do consent"


In [222]:
analysis_v2.to_csv("E29_CHECKIN1_MACROANALYSIS_V1.csv")